In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210607-1005-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [2]:
def get_reference_genome(
    locus: Union[hl.expr.LocusExpression, hl.expr.IntervalExpression],
    add_sequence: bool = False,
) -> hl.ReferenceGenome:
    """
    Returns the reference genome associated with the input Locus expression
    :param locus: Input locus
    :param add_sequence: If set, the fasta sequence is added to the reference genome
    :return: Reference genome
    """
    if isinstance(locus, hl.expr.LocusExpression):
        ref = locus.dtype.reference_genome
    else:
        assert isinstance(locus, hl.expr.IntervalExpression)
        ref = locus.dtype.point_type.reference_genome
    if add_sequence:
        ref = add_reference_sequence(ref)
    return ref

def filter_to_autosomes(
    t: Union[hl.MatrixTable, hl.Table]
) -> Union[hl.MatrixTable, hl.Table]:
    """
    Filters the Table or MatrixTable to autosomes only.
    This assumes that the input contains a field named `locus` of type Locus
    :param t: Input MT/HT
    :return:  MT/HT autosomes
    """
    reference = get_reference_genome(t.locus)
    autosomes = hl.parse_locus_interval(
        f"{reference.contigs[0]}-{reference.contigs[21]}", reference_genome=reference
    )
    return hl.filter_intervals(t, [autosomes])

In [3]:
mt=hl.read_matrix_table(f'{lustre_dir}/MegaWESSanger_cohorts_sampleQC_filtered.mt')

In [4]:
mt.locus.summarize()

Non-missing,28803842 (100.00%)
Missing,0
Contig Counts,"{'chr11': 1632197, 'chr5': 1342615, 'chr22': 696147, 'chr8': 1104562, 'chr19': 1662084, 'chrY': 28704, 'chr1': 2759565, 'chr15': 1077895, 'chr12': 1571754, 'chr18': 563132, 'chr20': 720998, 'chr2': 2099677, 'chr13': 596180, 'chr7': 1431748, 'chr14': 991416, 'chr3': 1653803, 'chrM': 7068, 'chr17': 1555676, 'chr4': 1200680, 'chr6': 1360406, 'chr9': 1159584, 'chrX': 760027, 'chr10': 1147841, 'chr21': 354289, 'chr16': 1325794}"


In [4]:
#mt = mt.key_rows_by('locus').distinct_by_row(
#    ).key_rows_by('locus', 'alleles')
   
    
mt=filter_to_autosomes(mt)

mt.write(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_sampleQC_filtered_autosomes.mt', overwrite=True)


FatalError: IOException: Mkdirs failed to create file:/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/MegaWESSanger_cohorts_sampleQC_filtered_autosomes.mt/rows/rows/parts (exists=false, cwd=file:/opt/spark/work/app-20210607100535-0058/43)

Java stack trace:
org.apache.spark.SparkException: Job aborted due to stage failure: Task 232 in stage 0.0 failed 4 times, most recent failure: Lost task 232.3 in stage 0.0 (TID 378, 192.168.252.89, executor 43): java.io.IOException: Mkdirs failed to create file:/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc/variant_qc/MegaWESSanger_cohorts_sampleQC_filtered_autosomes.mt/rows/rows/parts (exists=false, cwd=file:/opt/spark/work/app-20210607100535-0058/43)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:458)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:443)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1118)
	at org.apache.hadoop.fs.FileSystem.create(FileSyst